<img src="../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

## _*Classical Optimization*_ 

The latest version of this notebook is available on https://github.com/IBM/qiskit-sdk-py/tree/master/scripts.

For more information about how to use the IBM Q Experience (QX), consult the [Quantum Experience tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0), or check out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors
Jay Gambetta, Antonio Mezzacapo, Ramis Movassagh, Kristan Temme

## Introduction

Many problems in finance and optimization can be cast into classical Ising models. Here we show a method that is very similar to Ref. [Fahri] for finding the ground state of the Ising problem using short depth quantum circuits. 

Here we propose that we map the cost function of the problem to a Hamiltonian of the form

$$ H_\mathrm{Ising} = \alpha_i Z_i + \beta_{i,j} Z_jZ_i,$$

and the goal is to find the configuration that minimizes the energy.


Pseudo code

```
set the maximal control updates max_trials 
set initial controls u
loop through the max_trials
    prepare trial state |psi(u)> 
    
    Evaluate cost funtion C = <H> 
    
    Evaluate grad_u C (control update direction)
    
    update controls u -> u - grad_u C
    
    break if you find a good solution
    
return |<z|psi>|^2
```

The trial functions we propose using are of the form 

$$|\psi(\theta)\rangle  = [U_\mathrm{single}(\theta) U_\mathrm{entangler}]^m |+\rangle$$

where $U_\mathrm{entangler}$ is a function of cPhase gates, and $U_\mathrm{single}(\theta) = Y(\theta)^{\otimes n}$, where $n$ is the number of qubits and $m$ is the depth of the quantum circuit.

This can easily be extended: 

$$H = H_\mathrm{Ising}+ \gamma_{i,j,k}Z_kZ_jZ_i +... $$ 

Below we have shown how different problems can be mapped to the Ising model.

ADD

In [1]:
# Checking the version of PYTHON; we only support 3 at the moment
import sys
if sys.version_info < (3,0):
    raise Exception("Please use Python version 3 or greater.")
    
# useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import linalg as la

import sys
sys.path.append("../../")
# importing the QISKit
from qiskit import QuantumCircuit, QuantumProgram
import Qconfig

# import basic plot tools
from qiskit.basicplotter import plot_histogram

# import optimization tools
from tools.optimizationtools import trial_funtion_optimization
from tools.optimizationtools import cost_function, make_Hamiltonian

Enter the cost function below that you would like to minimize.

* [Max Cut](#sectionmaxcut)
 * [Two Qubits](#sectionmaxcuttwo)


## Max Cut<a id='sectionmaxcut'></a>

### Two Qubits<a id='sectionmaxcuttwo'></a>

In [2]:
# cost function H = alpha_i z_i + beta_ij z_i z_j
n=2 # number of qubits

alpha = np.zeros(n)
beta = np.zeros((n, n))
beta[0, 1] = 1

In [3]:
#Making the Hamiltonian in its full form and getting the lowest eigenvalue and egienvector
H = make_Hamiltonian(n,alpha,beta)
print(H)
w, v = la.eigh(H, eigvals=(0, 0))
print("Eigenvalue is %f and the state is" %(w))
print(v)

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]
Eigenvalue is -1.000000 and the state is
[[ 0.+0.j]
 [ 0.+0.j]
 [ 1.+0.j]
 [ 0.+0.j]]


In [4]:
# making a classical trial function for the problem 
device = 'local_qasm_simulator' # the device to run on
shots = 1024    # the number of shots in the experiment. 

Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken, Qconfig.config["url"]) # set the APIToken and API url

# Creating registers
q = Q_program.create_quantum_registers("q", n)
c = Q_program.create_classical_registers("c", n)

# Quantum circuit  
trial_circuit = Q_program.create_circuit("trial_exact", ["q"], ["c"])
trial_circuit.x(q[1])

for j in range (n):
    trial_circuit.measure(q[j], c[j])

circuits = ["trial_exact"]
Q_program.execute(circuits, device, shots)

# calculating the cost for this trial function
cost_function(Q_program.get_counts("trial_exact"),n,alpha,beta)

>> quantum_registers created: q 2
>> classical_registers created: c 2
running on backend: local_qasm_simulator


-1.0

In [ ]:
# quantum circuit parameters
device = 'local_qasm_simulator' # the device to run on
shots = 8192    # the number of shots in the experiment. 
entangler_map = {0: [1]} # the map of two-qubit gates with control at key and target at values

# Numerical parameters 
SPSA_parameters = np.array([.3,0.602,0,.1,0.101]) #[a, alpha, A, c, gamma]
max_trials = 100;
theta_depth_1 = np.random.randn(1*n) # initial controls 
theta_depth_2 = np.random.randn(2*n) # initial controls 
theta_depth_3 = np.random.randn(3*n) # initial controls 

cost_plus_depth_1=np.zeros(max_trials)
cost_minus_depth_1=np.zeros(max_trials)
cost_plus_depth_2=np.zeros(max_trials)
cost_minus_depth_2=np.zeros(max_trials)
cost_plus_depth_3=np.zeros(max_trials)
cost_minus_depth_3=np.zeros(max_trials)

for k in range(max_trials):
    if k % 10 == 0:
        print('trial ' + str(k) + " of " + str(max_trials))
    a_spsa = float(SPSA_parameters[0])/np.power(k+1+SPSA_parameters[2], SPSA_parameters[1])
    c_spsa = float(SPSA_parameters[3])/np.power(k+1, SPSA_parameters[4])

    Delta_depth_1 = 2*np.random.randint(2,size=n*1)-1 # \pm 1 random distribution 
    Delta_depth_2 = 2*np.random.randint(2,size=n*2)-1 # \pm 1 random distribution 
    Delta_depth_3 = 2*np.random.randint(2,size=n*3)-1 # \pm 1 random distribution 
    
    theta_plus_depth_1 = theta_depth_1 + c_spsa*Delta_depth_1
    theta_minus_depth_1 = theta_depth_1 - c_spsa*Delta_depth_1
    theta_plus_depth_2 = theta_depth_2 + c_spsa*Delta_depth_2
    theta_minus_depth_2 = theta_depth_2 - c_spsa*Delta_depth_2
    theta_plus_depth_3 = theta_depth_3 + c_spsa*Delta_depth_3
    theta_minus_depth_3 = theta_depth_3 - c_spsa*Delta_depth_3

    trial_circuit_plus_depth_1 = trial_funtion_optimization(n,1,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_1 = trial_funtion_optimization(n,1,theta_minus_depth_1,entangler_map) 
    trial_circuit_plus_depth_2 = trial_funtion_optimization(n,2,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_2 = trial_funtion_optimization(n,2,theta_minus_depth_2,entangler_map)
    trial_circuit_plus_depth_3 = trial_funtion_optimization(n,3,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_3 = trial_funtion_optimization(n,3,theta_minus_depth_3,entangler_map)
    
    Q_program.add_circuit("trial_circuit_plus_depth_1", trial_circuit_plus_depth_1)
    Q_program.add_circuit("trial_circuit_minus_depth_1", trial_circuit_minus_depth_1)
    Q_program.add_circuit("trial_circuit_plus_depth_2", trial_circuit_plus_depth_2)
    Q_program.add_circuit("trial_circuit_minus_depth_2", trial_circuit_minus_depth_2)
    Q_program.add_circuit("trial_circuit_plus_depth_3", trial_circuit_plus_depth_3)
    Q_program.add_circuit("trial_circuit_minus_depth_3", trial_circuit_minus_depth_3)
    
    circuits = ["trial_circuit_plus_depth_1", "trial_circuit_minus_depth_1","trial_circuit_plus_depth_2", "trial_circuit_minus_depth_2","trial_circuit_plus_depth_3", "trial_circuit_minus_depth_3"]
    Q_program.execute(circuits, device, shots)
    
    cost_plus_depth_1[k] = cost_function(Q_program.get_counts("trial_circuit_plus_depth_1"),n,alpha,beta)
    cost_minus_depth_1[k] = cost_function(Q_program.get_counts("trial_circuit_minus_depth_1"),n,alpha,beta)
    cost_plus_depth_2[k] = cost_function(Q_program.get_counts("trial_circuit_plus_depth_2"),n,alpha,beta)
    cost_minus_depth_2[k] = cost_function(Q_program.get_counts("trial_circuit_minus_depth_2"),n,alpha,beta)
    cost_plus_depth_3[k] = cost_function(Q_program.get_counts("trial_circuit_plus_depth_3"),n,alpha,beta)
    cost_minus_depth_3[k] = cost_function(Q_program.get_counts("trial_circuit_minus_depth_3"),n,alpha,beta)
    
    g_spsa_depth_1 = (cost_plus_depth_1[k]-cost_minus_depth_1[k])*Delta_depth_1/(2.0*c_spsa)
    g_spsa_depth_2 = (cost_plus_depth_2[k]-cost_minus_depth_2[k])*Delta_depth_2/(2.0*c_spsa)
    g_spsa_depth_3 = (cost_plus_depth_3[k]-cost_minus_depth_3[k])*Delta_depth_3/(2.0*c_spsa)

    theta_depth_1 = theta_depth_1 - a_spsa*g_spsa_depth_1
    theta_depth_2 = theta_depth_2 - a_spsa*g_spsa_depth_2
    theta_depth_3 = theta_depth_3 - a_spsa*g_spsa_depth_3

trial_circuit_depth_1 =  trial_funtion_optimization(n,1,theta_depth_1,entangler_map) 
trial_circuit_depth_2 =  trial_funtion_optimization(n,2,theta_depth_2,entangler_map) 
trial_circuit_depth_3 =  trial_funtion_optimization(n,3,theta_depth_3,entangler_map) 

Q_program.add_circuit("trial_circuit_depth_1", trial_circuit_depth_1)
Q_program.add_circuit("trial_circuit_depth_2", trial_circuit_depth_2)
Q_program.add_circuit("trial_circuit_depth_3", trial_circuit_depth_3)

circuits = ["trial_circuit_depth_1", "trial_circuit_depth_2","trial_circuit_depth_3"]
Q_program.execute(circuits, device, shots)

cost_depth_1 = cost_function(Q_program.get_counts("trial_circuit_depth_1"),n,alpha,beta)
cost_depth_2 = cost_function(Q_program.get_counts("trial_circuit_depth_2"),n,alpha,beta)
cost_depth_3 = cost_function(Q_program.get_counts("trial_circuit_depth_3"),n,alpha,beta)
print('m=1 ' + str(cost_depth_1) + ' m=2 ' + str(cost_depth_2) + 'm=3 ' + str(cost_depth_3))

trial 0 of 100
running on backend: local_qasm_simulator


In [ ]:
# plotting data
plt.plot(range(max_trials), cost_plus_depth_1, range(max_trials), cost_minus_depth_1)
plot_histogram(Q_program.get_counts("trial_circuit_depth_1"))
plt.plot(range(max_trials), cost_plus_depth_2, range(max_trials), cost_minus_depth_2)
plot_histogram(Q_program.get_counts("trial_circuit_depth_1"))
plt.plot(range(max_trials), cost_plus_depth_3, range(max_trials), cost_minus_depth_3)
plot_histogram(Q_program.get_counts("trial_circuit_depth_1"))

### Three Qubits<a id='sectionmaxcutthree'></a>

In [ ]:
# cost function H = alpha_i z_i + beta_ij z_i z_j
n = 3 
alpha = np.zeros(n)
beta = np.zeros((n, n))
beta[0, 1] = 1
beta[0, 2] = 1
beta[1, 2] = 1

In [ ]:
#Making the Hamiltonian in its full form and getting the lowest eigenvalue and egienvector
H = make_Hamiltonian(n,alpha,beta)
print(H)
w, v = la.eigh(H, eigvals=(0, 0))
print("Eigenvalue is %f and the state is" %(w))
print(v)

In [ ]:
# making a classical trial function for the problem 
device = 'local_qasm_simulator' # the device to run on
shots = 1024    # the number of shots in the experiment. 

Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken, Qconfig.config["url"]) # set the APIToken and API url

# Creating registers
q = Q_program.create_quantum_registers("q", n)
c = Q_program.create_classical_registers("c", n)

# Quantum circuit  
trial_circuit = Q_program.create_circuit("trial_exact", ["q"], ["c"])
trial_circuit.x(q[1])

for j in range (n):
    trial_circuit.measure(q[j], c[j])

circuits = ["trial_exact"]
Q_program.execute(circuits, device, shots)

# calculating the cost for this trial function
cost_function(Q_program.get_counts("trial_exact"),n,alpha,beta)

In [ ]:
# quantum circuit parameters
device = 'local_qasm_simulator' # the device to run on
shots = 8192    # the number of shots in the experiment. 
entangler_map = {0: [1]} # the map of two-qubit gates with control at key and target at values

# Numerical parameters 
SPSA_parameters = np.array([.3,0.602,0,.1,0.101]) #[a, alpha, A, c, gamma]
max_trials = 100;
theta_depth_1 = np.random.randn(1*n) # initial controls 
theta_depth_2 = np.random.randn(2*n) # initial controls 
theta_depth_3 = np.random.randn(3*n) # initial controls 

cost_plus_depth_1=np.zeros(max_trials)
cost_minus_depth_1=np.zeros(max_trials)
cost_plus_depth_2=np.zeros(max_trials)
cost_minus_depth_2=np.zeros(max_trials)
cost_plus_depth_3=np.zeros(max_trials)
cost_minus_depth_3=np.zeros(max_trials)

for k in range(max_trials):
    if k % 10 == 0:
        print('trial ' + str(k) + " of " + str(max_trials))
    a_spsa = float(SPSA_parameters[0])/np.power(k+1+SPSA_parameters[2], SPSA_parameters[1])
    c_spsa = float(SPSA_parameters[3])/np.power(k+1, SPSA_parameters[4])

    Delta_depth_1 = 2*np.random.randint(2,size=n*1)-1 # \pm 1 random distribution 
    Delta_depth_2 = 2*np.random.randint(2,size=n*2)-1 # \pm 1 random distribution 
    Delta_depth_3 = 2*np.random.randint(2,size=n*3)-1 # \pm 1 random distribution 
    
    theta_plus_depth_1 = theta_depth_1 + c_spsa*Delta_depth_1
    theta_minus_depth_1 = theta_depth_1 - c_spsa*Delta_depth_1
    theta_plus_depth_2 = theta_depth_2 + c_spsa*Delta_depth_2
    theta_minus_depth_2 = theta_depth_2 - c_spsa*Delta_depth_2
    theta_plus_depth_3 = theta_depth_3 + c_spsa*Delta_depth_3
    theta_minus_depth_3 = theta_depth_3 - c_spsa*Delta_depth_3

    trial_circuit_plus_depth_1 = trial_funtion_optimization(n,1,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_1 = trial_funtion_optimization(n,1,theta_minus_depth_1,entangler_map) 
    trial_circuit_plus_depth_2 = trial_funtion_optimization(n,2,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_2 = trial_funtion_optimization(n,2,theta_minus_depth_2,entangler_map)
    trial_circuit_plus_depth_3 = trial_funtion_optimization(n,3,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_3 = trial_funtion_optimization(n,3,theta_minus_depth_3,entangler_map)
    
    Q_program.add_circuit("trial_circuit_plus_depth_1", trial_circuit_plus_depth_1)
    Q_program.add_circuit("trial_circuit_minus_depth_1", trial_circuit_minus_depth_1)
    Q_program.add_circuit("trial_circuit_plus_depth_2", trial_circuit_plus_depth_2)
    Q_program.add_circuit("trial_circuit_minus_depth_2", trial_circuit_minus_depth_2)
    Q_program.add_circuit("trial_circuit_plus_depth_3", trial_circuit_plus_depth_3)
    Q_program.add_circuit("trial_circuit_minus_depth_3", trial_circuit_minus_depth_3)
    
    circuits = ["trial_circuit_plus_depth_1", "trial_circuit_minus_depth_1","trial_circuit_plus_depth_2", "trial_circuit_minus_depth_2","trial_circuit_plus_depth_3", "trial_circuit_minus_depth_3"]
    Q_program.execute(circuits, device, shots)
    
    cost_plus_depth_1[k] = cost_function(Q_program.get_counts("trial_circuit_plus_depth_1"),n,alpha,beta)
    cost_minus_depth_1[k] = cost_function(Q_program.get_counts("trial_circuit_minus_depth_1"),n,alpha,beta)
    cost_plus_depth_2[k] = cost_function(Q_program.get_counts("trial_circuit_plus_depth_2"),n,alpha,beta)
    cost_minus_depth_2[k] = cost_function(Q_program.get_counts("trial_circuit_minus_depth_2"),n,alpha,beta)
    cost_plus_depth_3[k] = cost_function(Q_program.get_counts("trial_circuit_plus_depth_3"),n,alpha,beta)
    cost_minus_depth_3[k] = cost_function(Q_program.get_counts("trial_circuit_minus_depth_3"),n,alpha,beta)
    
    g_spsa_depth_1 = (cost_plus_depth_1[k]-cost_minus_depth_1[k])*Delta_depth_1/(2.0*c_spsa)
    g_spsa_depth_2 = (cost_plus_depth_2[k]-cost_minus_depth_2[k])*Delta_depth_2/(2.0*c_spsa)
    g_spsa_depth_3 = (cost_plus_depth_3[k]-cost_minus_depth_3[k])*Delta_depth_3/(2.0*c_spsa)

    theta_depth_1 = theta_depth_1 - a_spsa*g_spsa_depth_1
    theta_depth_2 = theta_depth_2 - a_spsa*g_spsa_depth_2
    theta_depth_3 = theta_depth_3 - a_spsa*g_spsa_depth_3

trial_circuit_depth_1 =  trial_funtion_optimization(n,1,theta_depth_1,entangler_map) 
trial_circuit_depth_2 =  trial_funtion_optimization(n,2,theta_depth_2,entangler_map) 
trial_circuit_depth_3 =  trial_funtion_optimization(n,3,theta_depth_3,entangler_map) 

Q_program.add_circuit("trial_circuit_depth_1", trial_circuit_depth_1)
Q_program.add_circuit("trial_circuit_depth_2", trial_circuit_depth_2)
Q_program.add_circuit("trial_circuit_depth_3", trial_circuit_depth_3)

circuits = ["trial_circuit_depth_1", "trial_circuit_depth_2","trial_circuit_depth_3"]
Q_program.execute(circuits, device, shots)

cost_depth_1 = cost_function(Q_program.get_counts("trial_circuit_depth_1"),n,alpha,beta)
cost_depth_2 = cost_function(Q_program.get_counts("trial_circuit_depth_2"),n,alpha,beta)
cost_depth_3 = cost_function(Q_program.get_counts("trial_circuit_depth_3"),n,alpha,beta)
print('m=1 ' + str(cost_depth_1) + ' m=2 ' + str(cost_depth_2) + 'm=3 ' + str(cost_depth_3))

In [ ]:
# plotting data
plt.plot(range(max_trials), cost_plus_depth_1, range(max_trials), cost_minus_depth_1)
plot_histogram(get_data(results,0),n)
plt.plot(range(max_trials), cost_plus_depth_2, range(max_trials), cost_minus_depth_2)
plot_histogram(get_data(results,1),n)
plt.plot(range(max_trials), cost_plus_depth_3, range(max_trials), cost_minus_depth_3)
plot_histogram(get_data(results,2),n)

## 4 qubit max cut

In [ ]:
# cost function H = alpha_i z_i + beta_ij z_i z_j
n =4
alpha = np.zeros(n)
beta = np.zeros((n, n))
beta[0, 1] = 0.5
beta[1, 0] = 0.5
beta[0, 2] = 0.5
beta[2, 0] = 0.5
beta[1, 2] = 0.5
beta[2, 1] = 0.5
beta[1, 3] = 0.5
beta[3, 1] = 0.5
beta[2, 3] = 0.5
beta[3, 2] = 0.5

In [ ]:
# quantum circuit parameters
device = 'simulator' # the device to run on
shots = 8192    # the number of shots in the experiment 
entangler_map = {0: [1], 1: [2], 3: [2]} # the map of two-qubit gates with control at key and target at values

# Numerical parameters 
SPSA_parameters = np.array([.3,0.602,0,.1,0.101]) #[a, alpha, A, c, gamma]
max_trials = 100;
theta_depth_1 = np.random.randn(1*n) # initial controls 
theta_depth_2 = np.random.randn(2*n) # initial controls 
theta_depth_3 = np.random.randn(3*n) # initial controls 

cost_plus_depth_1=np.zeros(max_trials)
cost_minus_depth_1=np.zeros(max_trials)
cost_plus_depth_2=np.zeros(max_trials)
cost_minus_depth_2=np.zeros(max_trials)
cost_plus_depth_3=np.zeros(max_trials)
cost_minus_depth_3=np.zeros(max_trials)

for k in range(max_trials):
    print('trial ' + str(k) + " of " + str(max_trials))
    a_spsa = float(SPSA_parameters[0])/np.power(k+1+SPSA_parameters[2], SPSA_parameters[1])
    c_spsa = float(SPSA_parameters[3])/np.power(k+1, SPSA_parameters[4])

    Delta_depth_1 = 2*np.random.randint(2,size=n*1)-1 # \pm 1 random distribution 
    Delta_depth_2 = 2*np.random.randint(2,size=n*2)-1 # \pm 1 random distribution 
    Delta_depth_3 = 2*np.random.randint(2,size=n*3)-1 # \pm 1 random distribution 
    
    theta_plus_depth_1 = theta_depth_1 + c_spsa*Delta_depth_1
    theta_minus_depth_1 = theta_depth_1 - c_spsa*Delta_depth_1
    theta_plus_depth_2 = theta_depth_2 + c_spsa*Delta_depth_2
    theta_minus_depth_2 = theta_depth_2 - c_spsa*Delta_depth_2
    theta_plus_depth_3 = theta_depth_3 + c_spsa*Delta_depth_3
    theta_minus_depth_3 = theta_depth_3 - c_spsa*Delta_depth_3

    trial_circuit_plus_depth_1 = trial_funtion_optimization(n,1,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_1 = trial_funtion_optimization(n,1,theta_minus_depth_1,entangler_map) 
    trial_circuit_plus_depth_2 = trial_funtion_optimization(n,2,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_2 = trial_funtion_optimization(n,2,theta_minus_depth_2,entangler_map)
    trial_circuit_plus_depth_3 = trial_funtion_optimization(n,3,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_3 = trial_funtion_optimization(n,3,theta_minus_depth_3,entangler_map)
    
    program = [trial_circuit_plus_depth_1,trial_circuit_minus_depth_1,trial_circuit_plus_depth_2
               ,trial_circuit_minus_depth_2,trial_circuit_plus_depth_3,trial_circuit_minus_depth_3]
    out = run_program(program,api,device,shots,max_credits=3)
    results=combine_jobs([out['id']], api, wait=20, timeout=440)
    
    cost_plus_depth_1[k] = cost_classical(get_data(results,0),n,alpha,beta)
    cost_minus_depth_1[k] = cost_classical(get_data(results,1),n,alpha,beta)
    cost_plus_depth_2[k] = cost_classical(get_data(results,2),n,alpha,beta)
    cost_minus_depth_2[k] = cost_classical(get_data(results,3),n,alpha,beta)
    cost_plus_depth_3[k] = cost_classical(get_data(results,4),n,alpha,beta)
    cost_minus_depth_3[k] = cost_classical(get_data(results,5),n,alpha,beta)
    
    g_spsa_depth_1 = (cost_plus_depth_1[k]-cost_minus_depth_1[k])*Delta_depth_1/(2.0*c_spsa)
    g_spsa_depth_2 = (cost_plus_depth_2[k]-cost_minus_depth_2[k])*Delta_depth_2/(2.0*c_spsa)
    g_spsa_depth_3 = (cost_plus_depth_3[k]-cost_minus_depth_3[k])*Delta_depth_3/(2.0*c_spsa)

    theta_depth_1 = theta_depth_1 - a_spsa*g_spsa_depth_1
    theta_depth_2 = theta_depth_2 - a_spsa*g_spsa_depth_2
    theta_depth_3 = theta_depth_3 - a_spsa*g_spsa_depth_3

trial_circuit_depth_1 =  trial_funtion_optimization(n,1,theta_depth_1,entangler_map) 
trial_circuit_depth_2 =  trial_funtion_optimization(n,2,theta_depth_2,entangler_map) 
trial_circuit_depth_3 =  trial_funtion_optimization(n,3,theta_depth_3,entangler_map) 

program = [trial_circuit_depth_1,trial_circuit_depth_2,trial_circuit_depth_3]
out = run_program(program,api,device,shots,max_credits=3)
results=combine_jobs([out['id']], api, wait=20, timeout=440)  
cost_depth_1 = cost_classical(get_data(results,0),n,alpha,beta)
cost_depth_2 = cost_classical(get_data(results,1),n,alpha,beta)
cost_depth_3 = cost_classical(get_data(results,2),n,alpha,beta)
print('m=1 ' + str(cost_depth_1) + ' m=2 ' + str(cost_depth_2) + 'm=3 ' + str(cost_depth_3))

In [ ]:
# plotting data
plt.plot(range(max_trials), cost_plus_depth_1, range(max_trials), cost_minus_depth_1)
plot_histogram(get_data(results,0),n)
plt.plot(range(max_trials), cost_plus_depth_2, range(max_trials), cost_minus_depth_2)
plot_histogram(get_data(results,1),n)
plt.plot(range(max_trials), cost_plus_depth_3, range(max_trials), cost_minus_depth_3)
plot_histogram(get_data(results,2),n)

# 10 Qubit - Max Cut

In [ ]:
# cost function H = alpha_i z_i + beta_ij z_i z_j
n = 10
alpha = np.zeros(n)
beta = np.random.choice([0, 0.5], size=(n,n), p=[1./3, 2./3])
for i in range(n):
    for j in range(i):
        beta[j,i]=beta[i,j]
for i in range(n):
    beta[i,i]=0;

In [ ]:
# quantum circuit parameters
device = 'simulator' # the device to run on
shots = 8192    # the number of shots in the experiment 
entangler_map = {0: [1], 1: [2], 2: [3], 3: [4], 4: [5], 5: [6], 6: [7], 7: [8], 8: [9]} # the map of two-qubit gates with control at key and target at values

# Numerical parameters 
SPSA_parameters = np.array([.3,0.602,0,.1,0.101]) # [a, alpha, A, c, gamma]
max_trials = 100;
theta_depth_1 = np.random.randn(1*n) # initial controls 
theta_depth_2 = np.random.randn(2*n) # initial controls 
theta_depth_3 = np.random.randn(3*n) # initial controls 

cost_plus_depth_1=np.zeros(max_trials)
cost_minus_depth_1=np.zeros(max_trials)
cost_plus_depth_2=np.zeros(max_trials)
cost_minus_depth_2=np.zeros(max_trials)
cost_plus_depth_3=np.zeros(max_trials)
cost_minus_depth_3=np.zeros(max_trials)

for k in range(max_trials):
    print('trial ' + str(k) + " of " + str(max_trials))
    a_spsa = float(SPSA_parameters[0])/np.power(k+1+SPSA_parameters[2], SPSA_parameters[1])
    c_spsa = float(SPSA_parameters[3])/np.power(k+1, SPSA_parameters[4])

    Delta_depth_1 = 2*np.random.randint(2,size=n*1)-1 # \pm 1 random distribution 
    Delta_depth_2 = 2*np.random.randint(2,size=n*2)-1 # \pm 1 random distribution 
    Delta_depth_3 = 2*np.random.randint(2,size=n*3)-1 # \pm 1 random distribution 
    
    theta_plus_depth_1 = theta_depth_1 + c_spsa*Delta_depth_1
    theta_minus_depth_1 = theta_depth_1 - c_spsa*Delta_depth_1
    theta_plus_depth_2 = theta_depth_2 + c_spsa*Delta_depth_2
    theta_minus_depth_2 = theta_depth_2 - c_spsa*Delta_depth_2
    theta_plus_depth_3 = theta_depth_3 + c_spsa*Delta_depth_3
    theta_minus_depth_3 = theta_depth_3 - c_spsa*Delta_depth_3

    trial_circuit_plus_depth_1 = trial_funtion_optimization(n,1,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_1 = trial_funtion_optimization(n,1,theta_minus_depth_1,entangler_map) 
    trial_circuit_plus_depth_2 = trial_funtion_optimization(n,2,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_2 = trial_funtion_optimization(n,2,theta_minus_depth_2,entangler_map)
    trial_circuit_plus_depth_3 = trial_funtion_optimization(n,3,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_3 = trial_funtion_optimization(n,3,theta_minus_depth_3,entangler_map)
    
    program = [trial_circuit_plus_depth_1,trial_circuit_minus_depth_1,trial_circuit_plus_depth_2
               ,trial_circuit_minus_depth_2,trial_circuit_plus_depth_3,trial_circuit_minus_depth_3]
    out = run_program(program,api,device,shots,max_credits=3)
    results=combine_jobs([out['id']], api, wait=20, timeout=440)
    
    cost_plus_depth_1[k] = cost_classical(get_data(results,0),n,alpha,beta)
    cost_minus_depth_1[k] = cost_classical(get_data(results,1),n,alpha,beta)
    cost_plus_depth_2[k] = cost_classical(get_data(results,2),n,alpha,beta)
    cost_minus_depth_2[k] = cost_classical(get_data(results,3),n,alpha,beta)
    cost_plus_depth_3[k] = cost_classical(get_data(results,4),n,alpha,beta)
    cost_minus_depth_3[k] = cost_classical(get_data(results,5),n,alpha,beta)
    
    g_spsa_depth_1 = (cost_plus_depth_1[k]-cost_minus_depth_1[k])*Delta_depth_1/(2.0*c_spsa)
    g_spsa_depth_2 = (cost_plus_depth_2[k]-cost_minus_depth_2[k])*Delta_depth_2/(2.0*c_spsa)
    g_spsa_depth_3 = (cost_plus_depth_3[k]-cost_minus_depth_3[k])*Delta_depth_3/(2.0*c_spsa)

    theta_depth_1 = theta_depth_1 - a_spsa*g_spsa_depth_1
    theta_depth_2 = theta_depth_2 - a_spsa*g_spsa_depth_2
    theta_depth_3 = theta_depth_3 - a_spsa*g_spsa_depth_3

trial_circuit_depth_1 =  trial_funtion_optimization(n,1,theta_depth_1,entangler_map) 
trial_circuit_depth_2 =  trial_funtion_optimization(n,2,theta_depth_2,entangler_map) 
trial_circuit_depth_3 =  trial_funtion_optimization(n,3,theta_depth_3,entangler_map) 

program = [trial_circuit_depth_1,trial_circuit_depth_2,trial_circuit_depth_3]
out = run_program(program,api,device,shots,max_credits=3)
results=combine_jobs([out['id']], api, wait=20, timeout=440)  
cost_depth_1 = cost_classical(get_data(results,0),n,alpha,beta)
cost_depth_2 = cost_classical(get_data(results,1),n,alpha,beta)
cost_depth_3 = cost_classical(get_data(results,2),n,alpha,beta)
print('m=1 ' + str(cost_depth_1) + ' m=2 ' + str(cost_depth_2) + 'm=3 ' + str(cost_depth_3))

In [ ]:
# plotting data
plt.plot(range(max_trials), cost_plus_depth_1, range(max_trials), cost_minus_depth_1)
plot_histogram(get_data(results,0),n)
plt.plot(range(max_trials), cost_plus_depth_2, range(max_trials), cost_minus_depth_2)
plot_histogram(get_data(results,1),n)
plt.plot(range(max_trials), cost_plus_depth_3, range(max_trials), cost_minus_depth_3)
plot_histogram(get_data(results,2),n)

In [ ]:
# cost function H = alpha_i z_i + beta_ij z_i z_j
n = 10
alpha = np.zeros(n)
beta = np.random.choice([0, 0.5], size=(n,n), p=[1./3, 2./3])
for i in range(n):
    for j in range(i):
        beta[j,i]=beta[i,j]
for i in range(n):
    beta[i,i]=0;

In [ ]:
# quantum circuit parameters
device = 'simulator' # the device to run on
shots = 8192    # the number of shots in the experiment 
entangler_map = {0: [1], 1: [2], 2: [3], 3: [4], 4: [5], 5: [6], 6: [7], 7: [8], 8: [9]} # the map of two-qubit gates with control at key and target at values

# Numerical parameters 
SPSA_parameters = np.array([.3,0.602,0,.1,0.101]) #[a, alpha, A, c, gamma]
max_trials = 100;
theta_depth_1 = np.random.randn(1*n) # initial controls 
theta_depth_2 = np.random.randn(2*n) # initial controls 
theta_depth_3 = np.random.randn(3*n) # initial controls 

cost_plus_depth_1=np.zeros(max_trials)
cost_minus_depth_1=np.zeros(max_trials)
cost_plus_depth_2=np.zeros(max_trials)
cost_minus_depth_2=np.zeros(max_trials)
cost_plus_depth_3=np.zeros(max_trials)
cost_minus_depth_3=np.zeros(max_trials)

for k in range(max_trials):
    print('trial ' + str(k) + " of " + str(max_trials))
    a_spsa = float(SPSA_parameters[0])/np.power(k+1+SPSA_parameters[2], SPSA_parameters[1])
    c_spsa = float(SPSA_parameters[3])/np.power(k+1, SPSA_parameters[4])

    Delta_depth_1 = 2*np.random.randint(2,size=n*1)-1 # \pm 1 random distribution 
    Delta_depth_2 = 2*np.random.randint(2,size=n*2)-1 # \pm 1 random distribution 
    Delta_depth_3 = 2*np.random.randint(2,size=n*3)-1 # \pm 1 random distribution 
    
    theta_plus_depth_1 = theta_depth_1 + c_spsa*Delta_depth_1
    theta_minus_depth_1 = theta_depth_1 - c_spsa*Delta_depth_1
    theta_plus_depth_2 = theta_depth_2 + c_spsa*Delta_depth_2
    theta_minus_depth_2 = theta_depth_2 - c_spsa*Delta_depth_2
    theta_plus_depth_3 = theta_depth_3 + c_spsa*Delta_depth_3
    theta_minus_depth_3 = theta_depth_3 - c_spsa*Delta_depth_3

    trial_circuit_plus_depth_1 = trial_funtion_optimization(n,1,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_1 = trial_funtion_optimization(n,1,theta_minus_depth_1,entangler_map) 
    trial_circuit_plus_depth_2 = trial_funtion_optimization(n,2,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_2 = trial_funtion_optimization(n,2,theta_minus_depth_2,entangler_map)
    trial_circuit_plus_depth_3 = trial_funtion_optimization(n,3,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_3 = trial_funtion_optimization(n,3,theta_minus_depth_3,entangler_map)
    
    program = [trial_circuit_plus_depth_1,trial_circuit_minus_depth_1,trial_circuit_plus_depth_2
               ,trial_circuit_minus_depth_2,trial_circuit_plus_depth_3,trial_circuit_minus_depth_3]
    out = run_program(program,api,device,shots,max_credits=3)
    results=combine_jobs([out['id']], api, wait=20, timeout=440)
    
    cost_plus_depth_1[k] = cost_classical(get_data(results,0),n,alpha,beta)
    cost_minus_depth_1[k] = cost_classical(get_data(results,1),n,alpha,beta)
    cost_plus_depth_2[k] = cost_classical(get_data(results,2),n,alpha,beta)
    cost_minus_depth_2[k] = cost_classical(get_data(results,3),n,alpha,beta)
    cost_plus_depth_3[k] = cost_classical(get_data(results,4),n,alpha,beta)
    cost_minus_depth_3[k] = cost_classical(get_data(results,5),n,alpha,beta)
    
    g_spsa_depth_1 = (cost_plus_depth_1[k]-cost_minus_depth_1[k])*Delta_depth_1/(2.0*c_spsa)
    g_spsa_depth_2 = (cost_plus_depth_2[k]-cost_minus_depth_2[k])*Delta_depth_2/(2.0*c_spsa)
    g_spsa_depth_3 = (cost_plus_depth_3[k]-cost_minus_depth_3[k])*Delta_depth_3/(2.0*c_spsa)

    theta_depth_1 = theta_depth_1 - a_spsa*g_spsa_depth_1
    theta_depth_2 = theta_depth_2 - a_spsa*g_spsa_depth_2
    theta_depth_3 = theta_depth_3 - a_spsa*g_spsa_depth_3
    print(cost_minus_depth_3[k] +   cost_plus_depth_3[k])

trial_circuit_depth_1 =  trial_funtion_optimization(n,1,theta_depth_1,entangler_map) 
trial_circuit_depth_2 =  trial_funtion_optimization(n,2,theta_depth_2,entangler_map) 
trial_circuit_depth_3 =  trial_funtion_optimization(n,3,theta_depth_3,entangler_map) 

program = [trial_circuit_depth_1,trial_circuit_depth_2,trial_circuit_depth_3]
out = run_program(program,api,device,shots,max_credits=3)
results=combine_jobs([out['id']], api, wait=20, timeout=440)  
cost_depth_1 = cost_classical(get_data(results,0),n,alpha,beta)
cost_depth_2 = cost_classical(get_data(results,1),n,alpha,beta)
cost_depth_3 = cost_classical(get_data(results,2),n,alpha,beta)
print('m=1 ' + str(cost_depth_1) + ' m=2 ' + str(cost_depth_2) + 'm=3 ' + str(cost_depth_3))

In [ ]:
# plotting data
plt.plot(range(max_trials), cost_plus_depth_1, range(max_trials), cost_minus_depth_1)
plot_histogram(get_data(results,0),n)
plt.plot(range(max_trials), cost_plus_depth_2, range(max_trials), cost_minus_depth_2)
plot_histogram(get_data(results,1),n)
plt.plot(range(max_trials), cost_plus_depth_3, range(max_trials), cost_minus_depth_3)
plot_histogram(get_data(results,2),n)

## Max Cut (DONNA) 

In [ ]:
# cost function H = alpha_i z_i + beta_ij z_i z_j
n = 6
alpha = np.zeros(n)
beta = np.zeros((n, n))
beta[0, 1] = 93/2
beta[0, 4] = 17/2
beta[0, 5] = 51/2
beta[1, 0] = 93/2
beta[1, 5] = 13/2
beta[1, 2] = 77/2
beta[2, 1] = 77/2
beta[2, 3] = 31/2
beta[2, 5] = 23/2
beta[3,2] = 31/2
beta[3,4] = 7/2
beta[3,5] = 46/2
beta[4,3] = 7/2
beta[4,5] = 65/2
beta[4,0] = 17/2

beta[5,0] = 51/2
beta[5,1] = 13/2
beta[5,2] = 23/2
beta[5,3] = 46/2
beta[5,4] = 65/2

In [ ]:
# quantum circuit parameters
device = 'simulator' # the device to run on
shots = 8192    # the number of shots in the experiment 
entangler_map = {0: [1], 1: [2], 2: [3], 3: [4]} # the map of two-qubit gates with control at key and target at values

# Numerical parameters 
SPSA_parameters = np.array([3,0.602,0,.3,0.101]) #[a, alpha, A, c, gamma]
max_trials = 100;
theta_depth_1 = np.random.randn(1*n) # initial controls 
theta_depth_2 = np.random.randn(2*n) # initial controls 
theta_depth_3 = np.random.randn(3*n) # initial controls 

cost_plus_depth_1=np.zeros(max_trials)
cost_minus_depth_1=np.zeros(max_trials)
cost_plus_depth_2=np.zeros(max_trials)
cost_minus_depth_2=np.zeros(max_trials)
cost_plus_depth_3=np.zeros(max_trials)
cost_minus_depth_3=np.zeros(max_trials)

for k in range(max_trials):
    print('trial ' + str(k) + " of " + str(max_trials))
    a_spsa = float(SPSA_parameters[0])/np.power(k+1+SPSA_parameters[2], SPSA_parameters[1])
    c_spsa = float(SPSA_parameters[3])/np.power(k+1, SPSA_parameters[4])

    Delta_depth_1 = 2*np.random.randint(2,size=n*1)-1 # \pm 1 random distribution 
    Delta_depth_2 = 2*np.random.randint(2,size=n*2)-1 # \pm 1 random distribution 
    Delta_depth_3 = 2*np.random.randint(2,size=n*3)-1 # \pm 1 random distribution 
    
    theta_plus_depth_1 = theta_depth_1 + c_spsa*Delta_depth_1
    theta_minus_depth_1 = theta_depth_1 - c_spsa*Delta_depth_1
    theta_plus_depth_2 = theta_depth_2 + c_spsa*Delta_depth_2
    theta_minus_depth_2 = theta_depth_2 - c_spsa*Delta_depth_2
    theta_plus_depth_3 = theta_depth_3 + c_spsa*Delta_depth_3
    theta_minus_depth_3 = theta_depth_3 - c_spsa*Delta_depth_3

    trial_circuit_plus_depth_1 = trial_funtion_optimization(n,1,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_1 = trial_funtion_optimization(n,1,theta_minus_depth_1,entangler_map) 
    trial_circuit_plus_depth_2 = trial_funtion_optimization(n,2,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_2 = trial_funtion_optimization(n,2,theta_minus_depth_2,entangler_map)
    trial_circuit_plus_depth_3 = trial_funtion_optimization(n,3,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_3 = trial_funtion_optimization(n,3,theta_minus_depth_3,entangler_map)
    
    program = [trial_circuit_plus_depth_1,trial_circuit_minus_depth_1,trial_circuit_plus_depth_2
               ,trial_circuit_minus_depth_2,trial_circuit_plus_depth_3,trial_circuit_minus_depth_3]
    out = run_program(program,api,device,shots,max_credits=3)
    results=combine_jobs([out['id']], api, wait=20, timeout=440)
    
    cost_plus_depth_1[k] = cost_classical(get_data(results,0),n,alpha,beta)
    cost_minus_depth_1[k] = cost_classical(get_data(results,1),n,alpha,beta)
    cost_plus_depth_2[k] = cost_classical(get_data(results,2),n,alpha,beta)
    cost_minus_depth_2[k] = cost_classical(get_data(results,3),n,alpha,beta)
    cost_plus_depth_3[k] = cost_classical(get_data(results,4),n,alpha,beta)
    cost_minus_depth_3[k] = cost_classical(get_data(results,5),n,alpha,beta)
    
    g_spsa_depth_1 = (cost_plus_depth_1[k]-cost_minus_depth_1[k])*Delta_depth_1/(2.0*c_spsa)
    g_spsa_depth_2 = (cost_plus_depth_2[k]-cost_minus_depth_2[k])*Delta_depth_2/(2.0*c_spsa)
    g_spsa_depth_3 = (cost_plus_depth_3[k]-cost_minus_depth_3[k])*Delta_depth_3/(2.0*c_spsa)

    theta_depth_1 = theta_depth_1 - a_spsa*g_spsa_depth_1
    theta_depth_2 = theta_depth_2 - a_spsa*g_spsa_depth_2
    theta_depth_3 = theta_depth_3 - a_spsa*g_spsa_depth_3

trial_circuit_depth_1 =  trial_funtion_optimization(n,1,theta_depth_1,entangler_map) 
trial_circuit_depth_2 =  trial_funtion_optimization(n,2,theta_depth_2,entangler_map) 
trial_circuit_depth_3 =  trial_funtion_optimization(n,3,theta_depth_3,entangler_map) 

program = [trial_circuit_depth_1,trial_circuit_depth_2,trial_circuit_depth_3]
out = run_program(program,api,device,shots,max_credits=3)
results=combine_jobs([out['id']], api, wait=20, timeout=440)  
cost_depth_1 = cost_classical(get_data(results,0),n,alpha,beta)
cost_depth_2 = cost_classical(get_data(results,1),n,alpha,beta)
cost_depth_3 = cost_classical(get_data(results,2),n,alpha,beta)
print('m=1 ' + str(cost_depth_1) + ' m=2 ' + str(cost_depth_2) + 'm=3 ' + str(cost_depth_3))

In [ ]:
# plotting data
plt.plot(range(max_trials), cost_plus_depth_1, range(max_trials), cost_minus_depth_1)
plot_histogram(get_data(results,0),n)
plt.plot(range(max_trials), cost_plus_depth_2, range(max_trials), cost_minus_depth_2)
plot_histogram(get_data(results,1),n)
plt.plot(range(max_trials), cost_plus_depth_3, range(max_trials), cost_minus_depth_3)
plot_histogram(get_data(results,2),n)

## REAL

In [ ]:
# quantum circuit parameters
device = 'real' # the device to run on
shots = 8192    # the number of shots in the experiment
n = 2
entangler_map = {0: [1]} # the map of two-qubit gates with control at key and target at values

# Numerical parameters 
SPSA_parameters = np.array([.3,0.602,0,.3,0.101]) #[a, alpha, A, c, gamma]
max_trials = 100;
theta_depth_1 = np.random.randn(1*n) # initial controls 
theta_depth_2 = np.random.randn(2*n) # initial controls 
theta_depth_3 = np.random.randn(3*n) # initial controls 

cost_plus_depth_1=np.zeros(max_trials)
cost_minus_depth_1=np.zeros(max_trials)
cost_plus_depth_2=np.zeros(max_trials)
cost_minus_depth_2=np.zeros(max_trials)
cost_plus_depth_3=np.zeros(max_trials)
cost_minus_depth_3=np.zeros(max_trials)

for k in range(max_trials):
    print('trial ' + str(k) + " of " + str(max_trials))
    a_spsa = float(SPSA_parameters[0])/np.power(k+1+SPSA_parameters[2], SPSA_parameters[1])
    c_spsa = float(SPSA_parameters[3])/np.power(k+1, SPSA_parameters[4])

    Delta_depth_1 = 2*np.random.randint(2,size=n*1)-1 # \pm 1 random distribution 
    Delta_depth_2 = 2*np.random.randint(2,size=n*2)-1 # \pm 1 random distribution 
    Delta_depth_3 = 2*np.random.randint(2,size=n*3)-1 # \pm 1 random distribution 
    
    theta_plus_depth_1 = theta_depth_1 + c_spsa*Delta_depth_1
    theta_minus_depth_1 = theta_depth_1 - c_spsa*Delta_depth_1
    theta_plus_depth_2 = theta_depth_2 + c_spsa*Delta_depth_2
    theta_minus_depth_2 = theta_depth_2 - c_spsa*Delta_depth_2
    theta_plus_depth_3 = theta_depth_3 + c_spsa*Delta_depth_3
    theta_minus_depth_3 = theta_depth_3 - c_spsa*Delta_depth_3

    trial_circuit_plus_depth_1 = trial_funtion_optimization(n,1,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_1 = trial_funtion_optimization(n,1,theta_minus_depth_1,entangler_map) 
    trial_circuit_plus_depth_2 = trial_funtion_optimization(n,2,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_2 = trial_funtion_optimization(n,2,theta_minus_depth_2,entangler_map)
    trial_circuit_plus_depth_3 = trial_funtion_optimization(n,3,theta_plus_depth_3,entangler_map) 
    trial_circuit_minus_depth_3 = trial_funtion_optimization(n,3,theta_minus_depth_3,entangler_map)
    
    program = [trial_circuit_plus_depth_1,trial_circuit_minus_depth_1,trial_circuit_plus_depth_2
               ,trial_circuit_minus_depth_2,trial_circuit_plus_depth_3,trial_circuit_minus_depth_3]
    out = run_program(program,api,device,shots,max_credits=5)
    results=combine_jobs([out['id']], api, wait=20, timeout=480)
    
    cost_plus_depth_1[k] = cost_classical(get_data(results,0),n,alpha,beta)
    cost_minus_depth_1[k] = cost_classical(get_data(results,1),n,alpha,beta)
    cost_plus_depth_2[k] = cost_classical(get_data(results,2),n,alpha,beta)
    cost_minus_depth_2[k] = cost_classical(get_data(results,3),n,alpha,beta)
    cost_plus_depth_3[k] = cost_classical(get_data(results,4),n,alpha,beta)
    cost_minus_depth_3[k] = cost_classical(get_data(results,5),n,alpha,beta)
    
    g_spsa_depth_1 = (cost_plus_depth_1[k]-cost_minus_depth_1[k])*Delta_depth_1/(2.0*c_spsa)
    g_spsa_depth_2 = (cost_plus_depth_2[k]-cost_minus_depth_2[k])*Delta_depth_2/(2.0*c_spsa)
    g_spsa_depth_3 = (cost_plus_depth_3[k]-cost_minus_depth_3[k])*Delta_depth_3/(2.0*c_spsa)

    theta_depth_1 = theta_depth_1 - a_spsa*g_spsa_depth_1
    theta_depth_2 = theta_depth_2 - a_spsa*g_spsa_depth_2
    theta_depth_3 = theta_depth_3 - a_spsa*g_spsa_depth_3

trial_circuit_depth_1 =  trial_funtion_optimization(n,1,theta_depth_1,entangler_map) 
trial_circuit_depth_2 =  trial_funtion_optimization(n,2,theta_depth_2,entangler_map) 
trial_circuit_depth_3 =  trial_funtion_optimization(n,3,theta_depth_3,entangler_map) 

program = [trial_circuit_depth_1,trial_circuit_depth_2,trial_circuit_depth_3]
out = run_program(program,api,device,shots,max_credits=5)
results=combine_jobs([out['id']], api, wait=20, timeout=240)  
cost_depth_1 = cost_classical(get_data(results,0),n,alpha,beta)
cost_depth_2 = cost_classical(get_data(results,1),n,alpha,beta)
cost_depth_3 = cost_classical(get_data(results,2),n,alpha,beta)
print('m=1 ' + str(cost_depth_1) + ' m=2 ' + str(cost_depth_2) + 'm=3 ' + str(cost_depth_3))

In [ ]:
# plotting data
plt.plot(range(max_trials), cost_plus_depth_1, range(max_trials), cost_minus_depth_1)
plot_histogram(get_data(results,0),n)
plt.plot(range(max_trials), cost_plus_depth_2, range(max_trials), cost_minus_depth_2)
plot_histogram(get_data(results,1),n)
plt.plot(range(max_trials), cost_plus_depth_3, range(max_trials), cost_minus_depth_3)
plot_histogram(get_data(results,2),n)